<a href="https://colab.research.google.com/github/Jakeh33/Planning-and-Decision/blob/main/Planning_and_Decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 规划算法

---
《Planning Algorithm》- Steven M. LaValle
- [online](http://lavalle.pl/planning/book.html)



# 基本资料介绍

## 前言绪论
- 运动策略库 [MSL](http://lavalle.pl/msl/)
- 相关材料 
  - [Planning Algorithm](http://lavalle.pl/planning/)




### 1.1 规划（过程 $\to$ 结果）
- 机器人运动规划：通常忽略动力学和其他各种微分约束，主要关注物体的平动和转动
- 机器人轨迹规划：应用机器人运动规划算法的解及确定如何以遵守机械限制的方式沿着规划解移动
- 控制理论运动规划：构建非线性动力系统的输入，驱动系统从初始状态移动到目标位置
- 人工智能规划：更倾向于离散问题，解决类似魔方、拼图、积木等任务。通过定义有限行动集，应用于离散状态集合，通过给出相应的行动序列构造解
 


### 1.3 基本组成
- 状态：状态空间的大小（状态量个数）多数情况下很大导致无法显示表达
- 时间：规划问题设计随时间进行的决策序列
- 行动：规划产生对状态进行操作的行动（输入/控制）
- 初始和目标状态
- 准则：根据状态及行动编码一个规划的期望结果
  - 可行性：寻找能够到达目标状态的规划
  - 最优性：可行的基础上满足某种最优指标
- 规划：开环（时间序列），闭环（状态相关的函数），如果不能反馈状态，需要根据目前可用的信息推导状态（信息状态）

## 离散规划
最简单的规划问题，状态空间有限或可数无线，因此不需要几何模型或者微分方程来描述离散规划问题，且不考虑不确定性。即所有模型完全可知和可预测。

### 2.1 离散可行规划简介
- 可行规划问题可以被阐述为确定是否存在一个输入序列使得有限状态机最终报告一个所需状态。

#### 2.1.1 问题表述
- 所有可能的状态x的几个称为状态空间X，对于离线规划，重要的是集合可数
- 状态转移函数$x' = f(x,u)$，行动$u \in U(x)$作用于当前状态x后产生新状态x'
- 规划的目的就是找到一个有限行动序列将初始状态变换到目标状态
- 可以将状态和行动组合形成**状态转移图(state transition graph)**



#### 2.1.2 实例
例2.1 二维栅格运动
- 状态$x=(i,j)$，坐标位置
- 行动$U =\{(\pm 1,0),(0,\pm1)\}$
- 状态转移$f(x,u) = x+u$
- 如果增加阴影，表示障碍，即构建出迷宫

例2.2 魔方
- 状态空间：位置颜色的组合几何
- 行动：12个可能
- 任务：找到行动序列返回同色状态



### 2.2 可行规划的搜索
图搜索算法，能够跟踪已经访问过的状态并访问所有可达状态，并且无多余探索过程，递增构建搜索图$\mathcal G(V,E)$

基本步骤：
1. 初始化：E为空，V包含一些初始状态
2. 选择顶点：为扩展选择一个顶点$x_{cur}$（通过保持优先级队列）
3. 应用行动：得到一个新状态$x_{new}$
4. 向图中插入一条有向边：
5. 检查解：确定$\mathcal G$是否编码了一条从起点到终点的路径
6. 返回第2步


#### 2.2.1 一般前向搜索
- 未访问：初始时除初始状态外全部为未访问
- 不活动：已访问及下一个可能的状态均为不活动（对搜索不再产生贡献）
- 活动的(Q)：已经遇见的和一些未访问的相邻状态

通用伪代码模板
```
Q.insert(x_i) and mark x_i as visited
while Q is not empty do 
  Q.getFirst() -> x
  if x in X_goal:
    return Success
  for u in U(x):
    x' = f(x,u)
    ## 指针序列 x' -> x,构建规划序列 
    if x' not visited :
      mark x' as visited
      Q.insert(x')
    esle:
      Resolve duplicate x'
return Failure
```
- Q保存活动的状态，各种**搜索算法明显的区别在于采用不同的函数对Q进行排序**。
- 最简单的方式：先进先出队列
- 上述代码仅确定是否存在解，没有给出规划

#### 2.2.2 特殊前向搜索

##### 广度优先（breadth-first search）
- 先进先出队列
- 搜索边界一致扩大
- 保证发现的第一个解使用的步数最小


##### 深度优先（depth-first search）
- 后进先出的栈

##### Dijkstra算法
- 寻找图中单源最短路径
- 动态规划的一种特殊形式
- 假设图中的每条边都有一个相应的非负代价$l(e)$，是应用行动所付出的代价，可以用$l(x,u)$表示，代表从状态x应用行动u产生的代价。
- 优先级队列Q按照已付代价和C升序：对每个状态x，值$C^*(x)$为从初始状态x_i到x产生的最优已付代价。
- 重复遇到某状态x'，如果已在Q队列中，比较代价值C，如果更新后需要重新排序。
- 当x'从Q中移除，该状态变为不活动，可以确定最优$C^*$
- 为什么说队列第一个元素必然是最优？：
  - 由于元素在Q中是按照代价升序排列，而想要达到第一个元素的其他路径必将经过其他状态，而其他状态的代价值必然有更大的代价。
- 假设用Fibonacci堆来实现，运行时间$O(|V|lg|V|+|E|)，|V|,|E|$分别对应图论表示的顶点和边的数量


##### A*
- Dijkstra的扩展，通过对从给定状态到目标状态的代价进行启发式估计，试图减少所需探索状态的数量。
- C(x)是从$x_i \to x$的已付代价，G(x)表示从$x \to x_g$的尚需代价。
- 可用动态规划法递增地计算$C^*(x)$，但是无法提前知道真正的最优尚需代价
- 在某些应用场景中可以构建一个合理的估计代价，如迷宫
- 假设代价就是总步数，$(i,j) \to (i',j')$的估计值$|i'-i|+|j'-j|$，不考虑障碍物。
- 目的是计算一个尽可能接近最优尚需代价值的估计值，并且保证不大于最优尚需代价值，记为$\hat G^*(x)$
- 运行逻辑与Dijkstra基本一致，除了排序函数:根据$C^*(x')+\hat G^*(x')$，即从起点到终点的最优代价的估计值。
- 估计值越接近真值，越利于搜索，当$\hat G^* =0$时，退化为Dijkstra

##### 最佳优先（best-first search）
- 按照最优尚需代价的估计值进行排序
- 得到解不一定是最优的
- 能够加速运算，但是经常非常“贪婪”，会偏爱那些在搜索中"看起来好的"状态


##### 迭代深化
- 搜索树有大量的分支时（下一层比上一层有更多的顶点）
- 将深度优先搜索转为一种系统的搜索方法，寻找所有与起点距离在迭代值以内的状态
- 相比广度优先搜索在最坏情况下性能要略好
- 与A\*思路结合可以产生IDA\*算法

#### 2.2.3 其他方案

##### 后向搜索


```
Backward 
Q.insert(x_g) and mark x_g as visited
while Q is not empty:
  x' = Q.get_first()
  if x' = x_i:
    return Success
  for u^ in U^(-1)(x):
    x = f^(-1)(x',u^)
    if x not in visited:
      mark x as visited
      Q.insert(x)
    else:
      Resolve duplicate x
return Failure 
```





##### 双向搜索


```
BiDirectional
QI.insert(x_i) and mark x_i as visited
QG.insert(x_g) and mark x_g as visited
while QI not empty and QG not empty:
  if QI not empty:
    x ← QI.GetFirst()
    if x in visited from x_g:
      return SUCCESS
    forall u ∈ U(x):
      x′ ← f(x, u)
      if x′ not visited:
        Mark x′ as visited
        QI.insert(x′)
      else:
        Resolve duplicate x′
  if QG not empty:
    x′ ← QG.GetFirst()
    if x′in visited from x_i:
      return SUCCESS
    forall u^ ∈ U^(−1)(x'):
      x ← f^(−1)(x',u^)
      if x not visited:
        Mark x as visited
        QG.Insert(x)
      else:
        Resolve duplicate x
return FAILUR
```



### 2.3 离散最优规划
- 从三个方面进行扩展：
  - 用一个阶段索引来指示当前的规划步
  - 引入一个代价泛函，指示在执行过程中累积的代价
  - 引入一个终止，指示何时终止规划
- 几乎所有的问题都可以用最小化或者最大化来定义性能指标
- 新增符号表示：
  - $\pi_k$表示K步规划
  - K是一个规划的准确长度($u_1,...,u_k)$
  - L表示K步规划的代价泛函，F代表最后一个阶段$F=K+1$
  - $L(\pi_k)=\sum^k_{k=1} l(x_k,u_k)+l_F(x_F) $
  - $l_F$取决于是否达到目标，达到为0，否则为无穷
  - 通过定义$l_F$将可行性约束转换为直接地优化
  


#### 2.3.1 最优定长规划(固定长度最优路径）
- 最优性原则导致——值迭代算法(value iteration)
- 采用迭代方法在状态空间上计算最优尚需代价（或已付代价）函数，特殊情况下演变为Dijkstra算法。

##### 后向值迭代
- 寻找所有初始状态开始的最优规划
- 从短的最优规划中产生长的最优规划的关键在于在X上构造最优尚需代价函数。
- 第$1 \to F$中的第k步，令$G^*_k$表示在最优规划执行过程中从$k \to F$的累积代价（后向）
- 第一次迭代时，$G^*_F=l_F(x_F)=G^*_F(f(x_K,u_K))$（由于没有行动，立即能够得到最后阶段的代价）
- 第二次迭代时，$G^*_K = \min_{u_K} \{l(x_K,u_K) +l_F(x_F)\}$，对每一个状态$x_K$进行计算，计算了从$K到F=K+1$的最优规划
- 根据流程得到递归表达式
 $$G^*_k(x_k) = \min_{u_k} \{l(x_k,u_k)+G^*_{k+1}(x_{k+1}) \}$$
- 这种方式需要保存每个阶段、每个状态下满足递归式最小化的行动，需要$O(K|X|)$存储空间


##### 例2.3 五状态最优规划
- 问题描述：
  - ![有向图](http://lavalle.pl/planning/img420.gif)
  - $X=\{a,b,c,d,e\},k=4,x_I =a,X_G=d$
  - $a\to a =2;a\to b =2;b\to c =1;b\to d =4;c\to a =1;c\to d =1;d\to c=1;
  d\to e =1$

- 尚需代价函数

|  | a | b | c | d | e |
|:--:|:--:|:--:|:--:|:--:|:--:|
|$G^*_5$|$\infty$|$\infty$|$\infty$|0|$\infty$|
|$G^*_4$|$\infty$|4|1|$\infty$|$\infty$|
|$G^*_3$|6|2|$\infty$|2|$\infty$|
|$G^*_2$|4|6|3|$\infty$|$\infty$|
|$G^*_1$|6|4|5|4|$\infty$|

- 简单说明
  - G5对应最目标d，代价0
  - G4对应一步能够到d的b和c，分别为4和1，此时d为无穷，因为d不存在自回环
  - G3对应能够两步到d，即能够一步到b和c的a，b，d
  - 以此类推，得到最终结果，由于初始状态为a，则定步长条件下的代价为6

##### 前向值迭代
- 寻找到达X中所有状态的最优规划
- 定义$C^*_K$为从1到k的最优已付代价
- 迭代表达式
$$C^*_k(x_k) = \min_{u_k^{-1}} \{C^*_{k-1}(x_{k-1})+l(x_{k-1},u_{k-1}) \}$$
- 针对例2.3计算最优已付代价

|  | a | b | c | d | e |
|:--:|:--:|:--:|:--:|:--:|:--:|
|$C^*_1$|0|$\infty$|$\infty$|$\infty$|$\infty$|
|$C^*_2$|2|2|$\infty$|$\infty$|$\infty$|
|$C^*_3$|4|4|3|$\infty$|$\infty$|
|$C^*_4$|4|6|5|4|7|
|$C^*_5$|6|6|5|6|5|



#### 2.3.2 变长度最优规划
- 不定义K的值，因此L的定义域非常大
- 在$U(x)$中包含一个特殊终止行动$u_T$，如果$u_k=u_T$，那么该行动将永远重复继续，状态不会变化，代价不会累积。
- 即对所有$i \geq k,u_i =u_T,x_i =x_k,l(x_i,u_T)=0$
- 后向迭代：找到所有能够达到$X_G$的状态和最优代价
$$u^* = {\arg\min}_{u \in U(x)} \{ l(x,u)+G^*(f(x,u))\}$$
- 前向迭代：找到从$x_I$出发能达到的状态和最优代价
$${\arg\min}_{u^{-1} \in U^{-1}} \{ C^*(f^{-1}(x,u^{-1}))+l(f^{-1}(x,u^{-1}),u')\}$$

##### 例2.5 可变长度值迭代
- 根据新规则绘制有向图  
![带终止条件](http://lavalle.pl/planning/img494.gif)
- 后向迭代

|  | a | b | c | d | e |
|:--:|:--:|:--:|:--:|:--:|:--:|
|$G^*0$|$\infty$|$\infty$|$\infty$|0|$\infty$|
|$G^*_{-1}$|$\infty$|4|1|0|$\infty$|
|$G^*_{-2}$|6|2|1|0|$\infty$|
|$G^*_{-3}$|4|2|1|0|$\infty$|
|$G^*_{-4}$|4|2|1|0|$\infty$|
|$G^*$|4|2|1|0|$\infty$|

- 前向迭代 假设$x_I=b$

|  | a | b | c | d | e |
|:--:|:--:|:--:|:--:|:--:|:--:|
|$C^*_1$|$\infty$|0|$\infty$|$\infty$|$\infty$|
|$C^*_2$|$\infty$|0|1|4|$\infty$|
|$C^*_3$|2|0|1|2|5|
|$C^*_4$|2|0|1|2|3|
|$C^*$|2|0|1|2|3|



#### 2.3.3 再论Dijkstra算法
- 值迭代：在整个状态空间上的重复计算
- Dijkstra：在状态空间中只流动一次
- 前向值迭代经过一些变化可以得到Dijkstra：
  - 假设代价为非负
  - 不可达状态：值无穷大 - 未访问（Unvisited）
  - 最优代价平稳（已达最优不再改变）- 不活动的（dead）
  - Q队列中：代价为有限但可能非最优
- **研究值迭代的必要性**：
  - 某些问题维持排序的队列很expensive
  - 值迭代更易扩展到更广泛的问题：连续状态空间，随机最优规划，动态对策平衡点等
- 正确理解两种算法：确定在一个给定的问题中哪一种更合适
- Dijkstra是label-correcting算法族中的一种，相比一般前向搜索算法的主要区别在于如果发现更好的已付代价，那么允许状态再次成为活动的。
- label-correcting与标准前向搜索的另一个**重要的区别**在于使用目标状态的代价来删除一些候选路径，即代码第7行的if判断，因此只适用于单目标。经过适应性修改后能应用于多目标状态，但是性能会下降。

```
FORWARD_Label_Correcting(x_G):
  set C(s) = 10^5 for all x!=x_i 
  C(x_i) = 0
  Q.insert(x_i)
  while Q not empty:
    x =Q.getfirst()
    for u in U(x)
      x' = f(x,u)
      if C(x) +l(x,u) <min{C(x'),C(x_G)}
        % pointer x -> x'
        C(x') = C(x) +l(x,u)
        if(x' != x_G)
          Q.insert(x')
```

### 2.4 用逻辑来表示离散规划
- 构造离散规划的隐式表示，对于庞大的状态空间不必全部探索
- 能够很好地进行压缩，输出结果阐述了到达目标的步骤
- 但是很难进行一般化


#### 2.4.1 类似STRIPS的表示
- 最常见的逻辑表示
- 一个有限、非空个体（instance）集合I
- 一个有限、非空谓词（predicates）集合P
- 一个有限、非空算子集合O
- 初始集合S：正文字的一个集合
- 目标集合G：正文字和负文字的一个集合
- 正文字（positive literal）：谓词应用于个体的一个特定集合
- 负文字（negative literal）：正文字逻辑取反
- 互补对：表示正文字与其对应负文字

##### 例2.6 电池放入手电筒
- 实例分析上述抽象概念
- 问题：手电筒需要两节电池
- 个体：$I =\{Battery1,Battery2,Cap,Flashlight \}$
- 谓词：$On,In$:On只用来评价Cap是否在Flashlight上，记为$On(Cap,Flashlight)$
- In有两种应用方式：$In(Battery1,Flashlight),In(Battery2,Flashlight)$
- 谓词只是部分函数，可能不希望任何个体都成为谓词的变元
- 初始集合$S=\{On(Cap,Flashlight) \}$，基于S，假设$\neg In(Battery1,Flashlight),\neg In(Battery2,Flashlight)$成立。因此初始状态为后盖在手电筒上，电池在外面。
- S是正文字的集合，负文字是隐含的，任何不在S中的正文字，假设初始对应负文字成立
- 目标状态：$G= \{ On(Cap,Flashlight),In(Battery1,Flashlight),In(Battery2,Flashlight) \}$
- 算子集合   
![Operation](http://lavalle.pl/planning/img532.gif)
- 规划结果：$(RemoveCap,Insert(Battery1),Insert(Battery2),Place(Cap)$
- 采用逻辑表达的规划存在大量的复杂结果。


#### 2.4.2 转换到状态空间表示
- 能够帮助修改搜索方法
- 假设每个谓词有k个变元，任何个体都能够出现在每个变元中，即存在着$|P|I|^k$个互补对，对应于将个体代入所有谓词的所有变元。
- 为了表示状态，必须从每个互补对中选择一个正文字或负文字，通过给个体和谓词安排一个线性顺序，将组合编码为二进制串。
- 在例2.6中给定如下状态$(On(Cap,Flashlight),\neg In(Battery1,Flashlight),In(Battery2,Flashlight))$，用0代表负文字，1代表正文字，此状态可表示为$x =101$
- 能够区分所有可能状态的综述对应可能的比特串的集合，长度为$2^{|P|I|^k}$
- 将初始状态$x_I$编码，目标集合$X_G$是与正目标文字和负目标文字相一致的集合
- 对每一个状态，$U(x)$表示满足前提条件的算子的集合

### 2.5 基于逻辑的规划方法
- 描述的方法在保持问题的完整性的同时试图减小实际搜索复杂性
- 不同的方法在不同的应用场合性能会有不同的表现

#### 2.5.1 部分规划空间中的搜索
- 除了直接在X中进行搜索之外，可选的另外一种方法时构造部分规划
- 在部分规划中迭代完成所需要的的子目标，并保证没有影响求解的冲突发生
- 部分规划（partial plan）$\sigma$的定义
  - 需要应用的算子的合集$O_\sigma$（如果算子包含变量，那么这些变量可以被赋予特定的值或者只作为变量）
  - $O_\sigma$上的一个偏序关系$\prec_\sigma$，指定算子的逻辑顺序
  - 一个绑定约束集$B_\sigma$，指示算子间的某些变量必须有相同的值
  - 一个因果关系集$C_\sigma$，具有$(o_1,l,o_2)$的形式，表示为满足2的一个前提条件（1完成l）
- 例2.7 在上述手电筒的例子中，一个部分规划表示为
  - $O_\sigma = \{RemoveCap,Insert(Battery1)\}$
  - $RemoveCap \prec_\sigma Insert(Battery1)$
  - 因果关联C:$(RemoveCap,\neg On(Cap,Flashlight),Insert(Battery1))$
- Place-Space 规划
  - 从任意初始部分规划$\sigma$开始
  - 发现$\sigma$的一个缺陷，可能是
    - 没有实现的一个算子前提条件
    - $O_\sigma$中的一个威胁到$C_\sigma$中某个因果约束的孙子
    - 如果没有缺陷则为一个完整解，并计算满足所有约束的$O_\sigma$的线性排列
    - 如果缺陷是未实现的前提条件l，发现一个能够实现它的算子并记录为新的因果约束
    - 如果缺陷是对因果关联的威胁，那个通过更新$\prec_\sigma$来诱导产生一个合适的算子排序，或者更新$B_\sigma$
    - 返回第二步
  - 通过将一个部分规划延伸到另外一个离完成任务更近的部分规划来构造搜索图的边
  - 模板简单，但是依赖于初始规划的选择以及每次解决缺陷的方式

#### 2.5.2 建立规划图（planning graph）
- 引入规划图-数据结构，对哪些状态可达进行编码
- 从初始状态采用广度优先搜索（队列）
- 边L与算子O交替构成一个分层图（layered graph）
- $l_i 是o_{i-1}的结果，是o_i$的前提，算子中不包含变量
- 初始层包含S中的每个状态，不在S中的状态取反后也放入第一层$L_1$
- 构造平凡算子使得状态得以保持
- 手电筒的规划图  
![规划图](http://lavalle.pl/planning/img575.gif)
- $L_4 =L_3 ;0_4 =O_3$规划图稳定


#### 2.5.3 满足性规划
- 将规划问题转换成一个庞大的布尔满足性问题
- 通过确定对于产生TRUE值的表达式，变量是否有可能赋某个值

# 运动规划

## 几何表示与变换

### 3.1 几何建模

- 两种主要的建模表示方式
  - 边界表示：与表面大体一致的方程
  - 实体表示：包含实体中每个点的点集
- 定义世界$\mathcal W$
  - 二维世界 $\mathcal W= \mathbb R^2$
  - 三维世界 $\mathcal W= \mathbb R^3$
  - 包含两部分
    - 障碍物：不能通行的位置
    - 机器人：通过运动规划进行控制的物体
  - 障碍物和机器人都是$\mathcal W$的（闭）子集

  

#### 3.1.1 多边形与多面体模型
- 多边形模型
  - 边界由两个要素定义：顶点、边
- 用本原的组合来进行障碍区域$\mathcal O$的实体表示，用本原的有限数量的并集、交集、集差来定义
- 每个本原$H_i$代表$\mathcal W$的一个子集，一个复杂的障碍区域用有限个本源的布尔组合表示
- 凸多边形
  - 点集中任意两点连线上的点都在点集的内部
  $\lambda x_1 +(1-\lambda)x_2 \in X$
  - $H_i=\{(x,y)\in \mathcal W|f_i(x,y) \leq 0 \}$描述线$f_i(x,y)=0$的一侧所有点的集合的一个本原（多项式各项前系数的最大公约数为1）
  - $\mathcal O = H_1 \cap H_2 \cap ... \cap H_m $
- 非凸多边形
  - $\mathcal O = \mathcal O_1 \cap ... \cap \mathcal O_n$
- 定义逻辑谓词
  - $\phi$ 对应$\mathcal W \to \{TRUE,FALSE\} $，如果$\mathcal W$中的点在$\mathcal O$中，返TRUE。
  - $e(x,y)$如果$f(x,y)< 0$则返回TRUE；否则返回FALSE
  - 凸多边形对应的谓词表示$\alpha (x,y) = e_1(x,y) \wedge ... \wedge e_m(x,y)$
  - n个凸多边形组成的逻辑区域表示为$\phi(x,y) = \alpha_1(x,y) \vee ...\vee \alpha_n(x,y)$

- 多面体模型
  - 边界由三个要素定义：顶点、边、面
  - 顶点记录点的坐标以及指向任意与该顶点有接触的半边的指针
  - 半边是一条有向边
  - 每个面记录包含一个在它的边界上指向任意半边的指针（每个面由半边的循环列表构成）
  - 多面体的每条边都有一个有向半边记录的对  
  ![polyhedron](http://lavalle.pl/planning/img687.gif)


#### 3.1.2 半代数模型
- 由单个多项式本原确定的点集称为“代数集合”
- 由代数集合的有限的交并构成的点集称为半代数集

#### 3.1.3 其他模型
- 顶点按顺序编码，两点组成一个有向边，左边对应多边形内部，右边对应内部中空
- 图形学中，加速使用的是三角形，使用三角形组合成几何模型
- 不一致有理数B样条：在工程设计系统中应用较多，可以方便对曲面进行调整和设计
- 位图：将有限空间离散成矩形胞腔（单元），可以看作二值图，0表示不包含任何障碍区域的点。可以扩展成灰度图，在每个单元用数值表示一些事件的概率
- 超二次曲面
- 广义圆柱

### 3.2 刚体变换
- 令$\mathcal A$表示机器人
- $\mathcal O$在$\mathcal W$中固定
- $\mathcal A$在$\mathcal W$中运动

#### 3.2.1 一般概念
- 刚体变换$h:\mathcal A \to \mathcal W$，将机器人中的每一点映射到世界内
- 对于$a \in \mathcal A$，$h(a)$表示$\mathcal W$中被a占据的点
  $$h(\mathcal A) = \{h(a)\in \mathcal W | a\in \mathcal A\}$$
- 当模型用本原表示时，命名$w \in \mathcal W$，
$H(i) = \{ a \in \mathbb R | f_i(a) \leq 0 \}$ 
$h(H_i) = \{w \in \mathcal W|f_i(h^{-1}(w)) \leq 0 \} $
- 参数的变换族
  - 参数向量q
  - 点a
  - $h(q,a) \to a(q)$
- 坐标系
  - $\mathcal W$的原点和坐标基定义为世界坐标系
  - $\mathcal A$最初是在机体系下表达的

#### 3.2.2 二维变换
- 平移
  - $h(x,y) = (x+x_t,y+y_t)$,序列中的每个点$(x_i,y_i)$用$(x_i,y_i)$替代，得到变换后的表示
  $$h(H_i） = \{(x,y) \in \mathcal W |f(x-x_t,y-y_t)\leq 0\}$$
- 旋转 
  - $ R(\theta) = \begin{pmatrix} \cos \theta & -\sin \theta \\ \sin\theta &\cos \theta \end{pmatrix}$
- 组合变换
  - $ T= \begin{pmatrix} \cos \theta & -\sin \theta & x_t \\ \sin\theta &\cos \theta &y_t \\ 0&0&1 \end{pmatrix}$
  - 变换后的机器人可以用$\mathcal A(x_t,y_t,\theta)$表示

#### 3.2.3 三维变换
- 平移
  - $h(x,y,z) = (x+x_t,y+y_t,z+z_t)$
- 偏航、俯仰、滚转（欧拉旋转）
  - 偏航z轴
  $ R= \begin{pmatrix} \cos \alpha & -\sin \alpha & 0 \\ \sin\alpha &\cos \alpha &0 \\ 0&0&1 \end{pmatrix}$
  - 俯仰y轴$ R= \begin{pmatrix} \cos \beta & 0 & \sin \beta  \\ 0&1&0 \\ -\sin\beta &0&\cos \beta  \end{pmatrix}$
  - 滚转x轴$ R= \begin{pmatrix}   1&0&0\\ 0& \cos \gamma & -\sin \gamma  \\ 0& \sin\gamma &\cos \gamma  \end{pmatrix}$
  - $R(\alpha,\beta,\gamma)$，先滚转，后俯仰，最后偏航
- 任意旋转矩阵恢复欧拉角
  $$\alpha = atan2(r_{11},r_{21}) \\ 
  \beta = atan2(\sqrt{r_{32}^2+r_{33}^2},-r_{31}\\
  \gamma = atan2(r_{32},r_{33}) $$

### 3.3 物体运动链变换

#### 3.3.1 二维运动链
![two types](http://lavalle.pl/planning/img853.gif)
= 刚体在运动链中连接后，自由度会降低
- 对于旋转关节，一个连杆关于另一个连杆只能旋转 
- 对于滑动关节，一个连杆只能沿另一个连杆滑动
- 每种关节减少两个自由度
- 比如例子中的旋转关节，只有四个自由度$x_1,y_1,\theta_1,\theta_2$
- 每个独立连杆的体坐标系的原点都选择在$\mathcal A ,\mathcal A_i$之间的关节上，体系的x轴沿着刚体的方向
- $\mathcal A_i$的原点在$\mathcal A$的坐标系位置的变换矩阵$\alpha$是距离，$\theta$是方向差
$$T_i= \begin{bmatrix}\cos\theta_i &-\sin\theta_i &\alpha_{i-1} \\ \sin\theta_i &\cos\theta_i &0 \\ 0&0&1 \end{bmatrix} $$
- 任一点在$\mathcal W$的位置可以通过连乘变换矩阵得到$T_1T_2...T_m[x,y,1]^T$


#### 3.3.2 三维运动链
- 相比于二维，情况和选择更为复杂
- 但是整体思路还是可以沿用的
  - 仔细选择$\mathcal A_i$的体坐标系
  - 基于关节的位置，给定位置和角度参数
  - 定义一个变换矩阵
  - $\mathcal A_m$中点的坐标通过连乘变换矩阵得到
- 一般三维的旋转轴都是斜线
- 用$z_i$表示连接的旋转关节的旋转轴$x_i$表示连接两个相邻旋转轴的最短垂线，右手系确定$y_i$
- 三维情况下变换矩阵需要四个参数：$d_i,\theta_i,a_{i-1},\alpha_{i-1}$，即Denavit-Hartenberg（DH）参数。
  - $d_i$对应$x_{i-1}，x_i$在$z_i$上交点的有符号距离（跟随$z_i$轴的正负向）
  - $\theta_i$对应$x_{i-1} \to x_{i}$方向的角度
  - $a_{i-1}$是$z_{i-1}\to z_{i}$轴之间的距离
  - $\alpha_{i-1}$是$z_{i-1}\to z_{i}$轴之间的角度

![DH para](http://lavalle.pl/planning/img883.gif)
- 双螺旋体（关于同一个轴的旋转和平移的组合成为螺旋（类似螺钉上螺帽的运动））  
$$R_i = \begin{bmatrix}\cos\theta_i &-\sin\theta_i &0&0  \\ \sin\theta_i &\cos\theta_i &0 &0\\ 0&0&1&d_i \\ 0&0&0&1 \end{bmatrix} $$
  - 对应绕$z_i$旋转$\theta_i$、沿$z_i$平移$d_i$ 

$${Q_{i-1}} = \begin{bmatrix} 1&0&0&{a_{i-1}} \\ 0&\cos{\alpha_{i-1}} &-\sin{\alpha_{i-1}} &0  \\ 0& \sin{\alpha_{i-1}} &\cos\alpha_{i-1} &0 \\ 0&0&0&1\end{bmatrix} $$
  - 对应绕$x_{i-1}$轴的螺旋
- 变换矩阵$T = R_i Q_{i-1}$完成四步运算
  - 沿$z_i$平移$d_i$
  - 绕$z_i$逆时针旋转$\theta_i$
  - 沿${x_{i-1}}$轴平移$a_{i-1}$
  - 绕${x_{i-1}}$轴逆时针旋转$\alpha_{i-1}$
- 第一个变换矩阵$T_1$有6个自由度，就是完成的三维变换矩阵
- 经典的PUMA560机械臂为例

![puma](http://lavalle.pl/planning/img895.gif)  
![DH params](http://lavalle.pl/planning/img896.gif)

### 3.4 运动树的变换


- 目的
  - 连杆组成“树”，没有连成环，比如关节和四肢连在躯干上
  - 关节有旋转关节和球关节
  - 可以帮助在动画和虚拟环境中开发人型组件
  - 帮助生物化学分析分子结构
- 公共关节
  - 最简单的例子，有连杆的二维树
  - 多于两个旋转轴的联结（有超过两个关节的连杆）  
  ![](http://lavalle.pl/planning/img923.gif)
  - 如果只有一个联结，可以将其定为根，例如人体的躯干
  - 假设多个联结，在其中一个联结上定义的体坐标系有多种方法，一个链采用一种方法
- 约束参数  
![constraining param](http://lavalle.pl/planning/img934.gif)
  - 假设将$\theta_{7u}$作为独立变量，必须选择$\theta_{7u} = \theta_{7l} +\phi$
  - 就是将联结上的变换矩阵之间的角度形成约束
- 环形情况
  - 闭运动链：难以确定哪些参数值在其可接受范围内确保环的封闭
  - 如果能够确定参数范围，就能将闭环分解成两个树来进行变换
  - 然后使两个链的尾点坐标相等，得到一个非线性三角方程
  - 多数涉及环路的问题，得不到解集的合适参数表示，是著名的逆运动问题的一种形式
- 一般情况
  - 复杂的连杆排列可以想象为存在多个环路
  - 每忽略（断开）一个环路的关节就减少一个环路
  - 直到不存在环路为止
  - 每一个被断开的关节都能写出一个链路变换等式
  

### 3.5 非刚体的变换

- 其余变换
  - 在x轴和y轴放大倍数不同
  - 仿射变换
- 柔性材料
  - 弯曲一个柔性材料，比如使床垫弯成拱门
  - 往往可以采用参数化的曲线或者曲面
  - 样条模型是最合适的，因为它们通过调整较小数量的参数来实现曲线形状的控制。

## 位行空间